# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [165]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.ensemble import RandomForestRegressor

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [149]:
# Load the data
df_flights = pandas.read_csv('Data/flights_table.csv', index_col=0)

In [150]:
df_flights.columns

Index(['mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [121]:
df_flights.sample(1)

,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
fl_date,,,,,,,,,,,,,,,,,,,,,
2019-08-01,WN,WN,WN,2498,WN,N567WN,2498,11066,CMH,"Columbus, OH",...,829,1.0,0.0,2.0,0.0,41.0,NaN,NaN,NaN,NaN


In [122]:
df_features = df_flights[['mkt_unique_carrier','mkt_carrier_fl_num','origin','dest','crs_dep_time','crs_arr_time','distance','dep_delay','taxi_out','taxi_in', 'arr_delay','carrier_delay','weather_delay','nas_delay', 'security_delay','late_aircraft_delay']]

In [123]:
# count null values
df_features.isnull().sum()

mkt_unique_carrier         0
mkt_carrier_fl_num         0
origin                     0
dest                       0
crs_dep_time               0
crs_arr_time               0
distance                   0
dep_delay               1669
taxi_out                1719
taxi_in                 1767
arr_delay               1960
carrier_delay          81101
weather_delay          81101
nas_delay              81101
security_delay         81101
late_aircraft_delay    81101
dtype: int64

In [ ]:
# Let's take care of departure delay values that are null using mean value (10.585278)
df_features['dep_delay'].fillna(df['dep_delay'].mean(), inplace=True)

In [125]:
# count null values
df_features.isnull().sum()

mkt_unique_carrier         0
mkt_carrier_fl_num         0
origin                     0
dest                       0
crs_dep_time               0
crs_arr_time               0
distance                   0
dep_delay                  0
taxi_out                1719
taxi_in                 1767
arr_delay               1960
carrier_delay          81101
weather_delay          81101
nas_delay              81101
security_delay         81101
late_aircraft_delay    81101
dtype: int64

In [126]:
df_features[['crs_dep_time','crs_arr_time','distance','dep_delay','taxi_out','taxi_in', 'arr_delay','carrier_delay','weather_delay','nas_delay', 'security_delay','late_aircraft_delay']].describe()

,crs_dep_time,crs_arr_time,distance,dep_delay,taxi_out,taxi_in,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
count,100000.000000,100000.000000,100000.000000,100000.000000,98281.000000,98233.000000,98040.000000,18899.000000,18899.000000,18899.000000,18899.000000,18899.000000
mean,1330.118950,1483.993190,771.105830,10.585278,17.565735,7.679191,5.465239,19.802688,3.788507,16.501667,0.106619,27.593418
std,489.407088,516.814174,589.934162,48.588070,10.154389,6.162330,50.855428,62.464114,33.865191,37.880755,4.961265,55.970566
min,1.000000,1.000000,31.000000,-119.000000,0.000000,0.000000,-243.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,915.000000,1100.000000,339.000000,-6.000000,11.000000,4.000000,-15.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1320.000000,1512.000000,604.000000,-2.000000,15.000000,6.000000,-6.000000,0.000000,0.000000,3.000000,0.000000,3.000000
75%,1735.000000,1917.000000,1005.000000,8.000000,20.000000,9.000000,8.000000,17.000000,0.000000,20.000000,0.000000,34.000000
max,2359.000000,2400.000000,5095.000000,2976.000000,161.000000,248.000000,2973.000000,2973.000000,1302.000000,1183.000000,636.000000,1226.000000


### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

2. Statistics

In [153]:
df_features.describe()

,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,distance,dep_delay,taxi_out,taxi_in,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,day_of_week,week_of_year
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,98281.000000,98233.000000,98040.000000,18899.000000,18899.000000,18899.000000,18899.000000,18899.000000,100000.000000,100000.000000
mean,4.409550,2719.61824,181.179660,179.636450,1330.118950,1483.993190,771.105830,10.585278,17.565735,7.679191,5.465239,19.802688,3.788507,16.501667,0.106619,27.593418,2.935530,26.661710
std,3.789937,1852.26549,99.286836,99.885413,489.407088,516.814174,589.934162,48.588070,10.154389,6.162330,50.855428,62.464114,33.865191,37.880755,4.961265,55.970566,2.000763,14.814621
min,0.000000,0.00000,0.000000,0.000000,1.000000,1.000000,31.000000,-119.000000,0.000000,0.000000,-243.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,1114.00000,94.000000,92.000000,915.000000,1100.000000,339.000000,-6.000000,11.000000,4.000000,-15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,14.000000
50%,3.000000,2307.00000,192.000000,191.000000,1320.000000,1512.000000,604.000000,-2.000000,15.000000,6.000000,-6.000000,0.000000,0.000000,3.000000,0.000000,3.000000,3.000000,27.000000
75%,8.000000,4302.00000,264.000000,263.000000,1735.000000,1917.000000,1005.000000,8.000000,20.000000,9.000000,8.000000,17.000000,0.000000,20.000000,0.000000,34.000000,5.000000,39.000000
max,10.000000,6713.00000,369.000000,369.000000,2359.000000,2400.000000,5095.000000,2976.000000,161.000000,248.000000,2973.000000,2973.000000,1302.000000,1183.000000,636.000000,1226.000000,6.000000,52.000000


3. Converting certain alpha numeric columns to numeric using categorical encoding

In [154]:
df_features['origin'] = df_features['origin'].astype('category')
df_features['origin'] = df_features['origin'].cat.codes
df_features['dest'] = df_features['dest'].astype('category')
df_features['dest'] = df_features['dest'].cat.codes
df_features['mkt_unique_carrier'] = df_features['mkt_unique_carrier'].astype('category')
df_features['mkt_unique_carrier'] = df_features['mkt_unique_carrier'].cat.codes
df_features['mkt_carrier_fl_num'] = df_features['mkt_carrier_fl_num'].astype('category')
df_features['mkt_carrier_fl_num'] = df_features['mkt_carrier_fl_num'].cat.codes

4. Time of the day

In [ ]:
# convert index to datetime
df_features.index = pd.to_datetime(df_features.index)

# from df_features.index create a new column with the day of the week (0 = Monday, 6 = Sunday)
df_features['day_of_week'] = df_features.index.dayofweek

df_features['week_of_year'] = df_features.index.weekofyear

In [129]:
X = df_features[['mkt_unique_carrier','mkt_carrier_fl_num','origin','dest','crs_dep_time','crs_arr_time','distance']]

In [160]:
X = df_features[['origin', 'dest', 'crs_dep_time','crs_arr_time','distance']]
y = df_features['dep_delay']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state = 646)

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

Linear Regression

In [158]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [159]:
# Evaluate the model performance
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('R-squared:', r2)
print('Mean squared error:', mse)
print('Mean absolute error:', mae)

R-squared: 0.010354918107392974
Mean squared error: 2334.56103456066
Mean absolute error: 21.093920766343967


Random Forest

In [166]:
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, random_state=42)

In [167]:
y_pred = rf.predict(X_test)

# Evaluate the model performance
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print('R-squared:', r2)
print('Mean squared error:', mse)

R-squared: -0.0033714744660608975
Mean squared error: 2366.941432173292


### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.